In [1]:
# The goal of this notebook is dividing the dataset daily_consumption_census.csv for further cleaning

In [2]:
# Necessary imports to read the datasets
import pandas as pd
import zipfile
import os
import numpy as np

In [3]:
# We read the dataset
df = pd.read_csv("../data/datasets/daily_consumption_census.csv", low_memory=False)


In [4]:
df

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
0,0801501001,01,BADALONA,2021-05-26,Comercial/Comercial/Commercial,12,843
1,0801501001,01,BADALONA,2021-05-26,Domèstic/Doméstico/Domestic,161,4891
2,0801501001,01,BADALONA,2021-05-27,Comercial/Comercial/Commercial,12,2173
3,0801501001,01,BADALONA,2021-05-27,Domèstic/Doméstico/Domestic,173,15458
4,0801501001,01,BADALONA,2021-05-28,Comercial/Comercial/Commercial,12,1836
...,...,...,...,...,...,...,...
3547930,NaN,NaN,L'HOSPITALET LLOBR.,2023-12-30,Domèstic/Doméstico/Domestic,318,4199
3547931,NaN,NaN,L'HOSPITALET LLOBR.,2023-12-30,Industrial/Industrial/Industrial,12,25802
3547932,NaN,NaN,L'HOSPITALET LLOBR.,2023-12-31,Comercial/Comercial/Commercial,8,1277
3547933,NaN,NaN,L'HOSPITALET LLOBR.,2023-12-31,Domèstic/Doméstico/Domestic,318,5046


In [5]:
# Check for duplicates to ensure time series can be uniquely identified
dupes = df.duplicated(subset=["Census section", "District", "Municipality", "Use", "Number of meters", "Date"], keep=False)
duplicated_rows = df[dupes]
num_duplicates = duplicated_rows.shape[0]
print(f"Number of duplicated rows: {num_duplicates}")


Number of duplicated rows: 0


In [7]:
# Check for duplicates without considering 'Use'
dupes_no_use = df.duplicated(subset=["Census section", "District", "Municipality", "Number of meters", "Date"], keep=False)
duplicated_rows_no_use = df[dupes_no_use]
num_duplicates_no_use = duplicated_rows_no_use.shape[0]
print(f"Number of duplicated rows (excluding 'Use'): {num_duplicates_no_use}")


Number of duplicated rows (excluding 'Use'): 3642


A considerable number of rows in the dataset have missing values in the *Census Section* field. These missing entries likely represent unidentified or unrecorded census sections. As a result, multiple time series may be incorrectly grouped under the same identifier, even though they do not belong to the same geographical area. Since these cases represent a small and non-significant portion of the overall dataset, we choose to remove them at this stage to avoid introducing noise into the analysis.

In [9]:
print(df.isna().sum())         # Should print 0s for all columns
print(df.shape)                # To see how many rows remain

Census section                     9420
District                           9420
Municipality                          0
Date                                  0
Use                                   0
Number of meters                      0
Accumulated consumption (L/day)       0
dtype: int64
(3547935, 7)


In [10]:
df = df.dropna()

In [11]:
df

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
0,0801501001,01,BADALONA,2021-05-26,Comercial/Comercial/Commercial,12,843
1,0801501001,01,BADALONA,2021-05-26,Domèstic/Doméstico/Domestic,161,4891
2,0801501001,01,BADALONA,2021-05-27,Comercial/Comercial/Commercial,12,2173
3,0801501001,01,BADALONA,2021-05-27,Domèstic/Doméstico/Domestic,173,15458
4,0801501001,01,BADALONA,2021-05-28,Comercial/Comercial/Commercial,12,1836
...,...,...,...,...,...,...,...
3538510,<NULL>,>,L'HOSPITALET LLOBR.,2023-12-30,Domèstic/Doméstico/Domestic,518,13546
3538511,<NULL>,>,L'HOSPITALET LLOBR.,2023-12-30,Industrial/Industrial/Industrial,47,81874
3538512,<NULL>,>,L'HOSPITALET LLOBR.,2023-12-31,Comercial/Comercial/Commercial,32,5210
3538513,<NULL>,>,L'HOSPITALET LLOBR.,2023-12-31,Domèstic/Doméstico/Domestic,518,16331


In [12]:
print(df.isna().sum())         # Should print 0s for all columns
print(df.shape)                # To see how many rows remain

Census section                     0
District                           0
Municipality                       0
Date                               0
Use                                0
Number of meters                   0
Accumulated consumption (L/day)    0
dtype: int64
(3538515, 7)


We still have rows that have the value `<NULL>` we have to delete.

In [14]:
df = df[~df.isin(["<NULL>"]).any(axis=1)]

In [16]:
print(df[df.eq("<NULL>").any(axis=1)])  # Should return empty DataFrame

Empty DataFrame
Columns: [Census section, District, Municipality, Date, Use, Number of meters, Accumulated consumption (L/day)]
Index: []


In [17]:
df

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
0,0801501001,01,BADALONA,2021-05-26,Comercial/Comercial/Commercial,12,843
1,0801501001,01,BADALONA,2021-05-26,Domèstic/Doméstico/Domestic,161,4891
2,0801501001,01,BADALONA,2021-05-27,Comercial/Comercial/Commercial,12,2173
3,0801501001,01,BADALONA,2021-05-27,Domèstic/Doméstico/Domestic,173,15458
4,0801501001,01,BADALONA,2021-05-28,Comercial/Comercial/Commercial,12,1836
...,...,...,...,...,...,...,...
3531940,0822101006,01,BADALONA,2023-12-27,Domèstic/Doméstico/Domestic,9,149
3531941,0822101006,01,BADALONA,2023-12-28,Domèstic/Doméstico/Domestic,9,177
3531942,0822101006,01,BADALONA,2023-12-29,Domèstic/Doméstico/Domestic,9,147
3531943,0822101006,01,BADALONA,2023-12-30,Domèstic/Doméstico/Domestic,9,167


As we have seen, the data from 2021 may be noisy because we were in lockdown during the pandemic.

In [19]:
# As we know the data is from 2021, 2022, 2023
# 2021 is a noisy year for our predictions
df = df.copy()
df["Date"] = pd.to_datetime(df["Date"])

unique_years = df["Date"].dt.year.unique()
print(sorted(unique_years))

[2021, 2022, 2023]


In [20]:
# Keep only rows NOT from the year 2021
df = df[df["Date"].dt.year != 2021]

In [21]:
# Let's change the names of the Municipalities for an easier acces
municipality_map = {
    'BADALONA': 'Badalona',
    'BARCELONA': 'Barcelona',
    "L'HOSPITALET LLOBR.": 'Hospitalet'
}

# We map the new names
df.loc[:, "Municipality"] = df["Municipality"].replace(municipality_map)


### Observation
We observe that it is important to take the **Use** attribute into account. Therefore, we will split the dataset into three separate files, each corresponding to a different usage type. This is necessary, as each category may require different treatment during the preprocessing and modeling stages.


In [23]:
# We divide the datasets
# Create Domestic DataFrame
df_domestic = df[df["Use"] == "Domèstic/Doméstico/Domestic"].copy()
df_domestic["Use"] = "Domestic"

# Create Commercial DataFrame
df_comercial = df[df["Use"] == "Comercial/Comercial/Commercial"].copy()
df_comercial["Use"] = "Commercial"

# Create Industrial DataFrame
df_industrial = df[df["Use"] == "Industrial/Industrial/Industrial"].copy()
df_industrial["Use"] = "Industrial"


In [24]:
df_domestic

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
441,0801501001,01,Badalona,2022-01-01,Domestic,250,16242
443,0801501001,01,Badalona,2022-01-02,Domestic,251,17477
445,0801501001,01,Badalona,2022-01-03,Domestic,251,16540
447,0801501001,01,Badalona,2022-01-04,Domestic,251,17153
449,0801501001,01,Badalona,2022-01-05,Domestic,251,18619
...,...,...,...,...,...,...,...
3531940,0822101006,01,Badalona,2023-12-27,Domestic,9,149
3531941,0822101006,01,Badalona,2023-12-28,Domestic,9,177
3531942,0822101006,01,Badalona,2023-12-29,Domestic,9,147
3531943,0822101006,01,Badalona,2023-12-30,Domestic,9,167


In [25]:
df_comercial

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
440,0801501001,01,Badalona,2022-01-01,Commercial,21,228
442,0801501001,01,Badalona,2022-01-02,Commercial,21,825
444,0801501001,01,Badalona,2022-01-03,Commercial,21,1389
446,0801501001,01,Badalona,2022-01-04,Commercial,21,1397
448,0801501001,01,Badalona,2022-01-05,Commercial,21,1851
...,...,...,...,...,...,...,...
3528599,0819401002,01,Badalona,2023-12-27,Commercial,4,89
3528601,0819401002,01,Badalona,2023-12-28,Commercial,4,100
3528603,0819401002,01,Badalona,2023-12-29,Commercial,4,43
3528605,0819401002,01,Badalona,2023-12-30,Commercial,4,150


In [26]:
df_industrial

,Census section,District,Municipality,Date,Use,Number of meters,Accumulated consumption (L/day)
942,0801501001,01,Badalona,2022-09-08,Industrial,2,2768
945,0801501001,01,Badalona,2022-09-09,Industrial,2,4068
948,0801501001,01,Badalona,2022-09-10,Industrial,2,5654
951,0801501001,01,Badalona,2022-09-11,Industrial,2,4813
954,0801501001,01,Badalona,2022-09-12,Industrial,2,2726
...,...,...,...,...,...,...,...
3529875,0819403001,03,Barcelona,2023-09-23,Industrial,2,2755
3529877,0819403001,03,Barcelona,2023-09-24,Industrial,2,5699
3529879,0819403001,03,Barcelona,2023-09-25,Industrial,2,3126
3529881,0819403001,03,Barcelona,2023-09-26,Industrial,2,9193


In [29]:
# Columns to check uniqueness
key_columns = ["Census section", "Municipality", "Date"]

# Check duplicates in each dataset
domestic_duplicates = df_domestic.duplicated(subset=key_columns, keep=False)
comercial_duplicates = df_comercial.duplicated(subset=key_columns, keep=False)
industrial_duplicates = df_industrial.duplicated(subset=key_columns, keep=False)

# Count how many duplicates exist
print("Domestic duplicates:", domestic_duplicates.sum())
print("Commercial duplicates:", comercial_duplicates.sum())
print("Industrial duplicates:", industrial_duplicates.sum())

Domestic duplicates: 0
Commercial duplicates: 0
Industrial duplicates: 0


It's important to remark that District is redundant, it's enough Census Sections and Municipality for the identification.

In [31]:
# We identify the time series
for dataframe in [df_domestic, df_comercial, df_industrial]:
    dataframe["id"] = dataframe["Census section"].astype(str) + "_" + dataframe["Municipality"].astype(str)
    dataframe.drop(columns=["Census section", "Municipality"], inplace=True)

In [35]:
# Expected length of a complete time series (daily data for 2022 and 2023)
expected_length = 365 * 2  # 730

# Function to count time series and check their lengths
def analyze_time_series(df, label):
    # Count total unique time series (by 'id')
    unique_ids = df['id'].nunique()
    
    # Group by 'id' and count the number of entries per time series
    lengths = df.groupby('id').size()

    # Count how many time series have the expected length
    complete_series = (lengths == expected_length).sum()
    incomplete_series = (lengths != expected_length).sum()

    print(f"--- {label} ---")
    print(f"Unique time series (id): {unique_ids}")
    print(f"Time series with {expected_length} entries: {complete_series}")
    print(f"Time series with incorrect length: {incomplete_series}")
    print(f"Percentage of complete time series: {round(100 * complete_series / unique_ids, 2)}%")
    print()

# Analyze each DataFrame
analyze_time_series(df_domestic, "Domestic")
analyze_time_series(df_comercial, "Commercial")
analyze_time_series(df_industrial, "Industrial")


--- Domestic ---
Unique time series (id): 1419
Time series with 730 entries: 1415
Time series with incorrect length: 4
Percentage of complete time series: 99.72%

--- Commercial ---
Unique time series (id): 1400
Time series with 730 entries: 1382
Time series with incorrect length: 18
Percentage of complete time series: 98.71%

--- Industrial ---
Unique time series (id): 693
Time series with 730 entries: 434
Time series with incorrect length: 259
Percentage of complete time series: 62.63%



We can see most of the time series in the domestic df are complete.

In [44]:
# Filter rows where "Number of meters" is less than 10
low_meter_df = df_domestic[df_domestic["Number of meters"] < 10]

# Get the unique IDs of these time series
low_meter_ids = low_meter_df["id"].unique()

print(f"Number of time series with 'Number of meters' < 10: {len(low_meter_ids)}")
print(f"IDs: {low_meter_ids}")


Number of time series with 'Number of meters' < 10: 9
IDs: ['0801503013_Badalona' '0801509006_Badalona' '0801909008_Barcelona'
 '0801909031_Barcelona' '0807710008_Hospitalet' '0810102034_Barcelona'
 '0812505003_Barcelona' '0820506001_Barcelona' '0822101006_Badalona']


In [55]:
# Count the number of rows per time series (id)
lengths = low_meter_df.groupby("id").size().sort_values()

print("\nLengths of time series with <10 meters:")
print(lengths)



Lengths of time series with <10 meters:
id
0801909031_Barcelona      13
0801909008_Barcelona      19
0801509006_Badalona       52
0801503013_Badalona      224
0812505003_Barcelona     297
0810102034_Barcelona     627
0820506001_Barcelona     638
0807710008_Hospitalet    730
0822101006_Badalona      730
dtype: int64


We have two complete time series with few meters only.

Let's save the df

In [59]:
# The id column shoud be the first
def reorder_and_reset(df):
    df = df.reset_index(drop=True)
    cols = df.columns.tolist()
    if "id" in cols:
        cols.insert(0, cols.pop(cols.index("id")))
    return df[cols]

df_domestic_processed = reorder_and_reset(df_domestic)
df_comercial_processed = reorder_and_reset(df_comercial)
df_industrial_processed = reorder_and_reset(df_industrial)

# We save the csv files
df_domestic_processed.to_csv("../data/datasets/01_cleaned_daily/df_domestic.csv", index=False)
df_comercial_processed.to_csv("../data/datasets/01_cleaned_daily/df_comercial.csv", index=False)
df_industrial_processed.to_csv("../data/datasets/01_cleaned_daily/df_industrial.csv", index=False)


In [60]:
# We will continue working with this df
df_domestic_processed

,id,District,Date,Use,Number of meters,Accumulated consumption (L/day)
0,0801501001_Badalona,01,2022-01-01,Domestic,250,16242
1,0801501001_Badalona,01,2022-01-02,Domestic,251,17477
2,0801501001_Badalona,01,2022-01-03,Domestic,251,16540
3,0801501001_Badalona,01,2022-01-04,Domestic,251,17153
4,0801501001_Badalona,01,2022-01-05,Domestic,251,18619
...,...,...,...,...,...,...
1034644,0822101006_Badalona,01,2023-12-27,Domestic,9,149
1034645,0822101006_Badalona,01,2023-12-28,Domestic,9,177
1034646,0822101006_Badalona,01,2023-12-29,Domestic,9,147
1034647,0822101006_Badalona,01,2023-12-30,Domestic,9,167
